In [12]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

In [14]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [15]:
prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

In [16]:
output_parser = StrOutputParser()

In [17]:
# 체인
chain = prompt | llm | output_parser

In [18]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [19]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [21]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "소놀코딩은 어떤 회사인가요?"})

'기타'

In [23]:
# 세 개의 하위 체인을 생성

In [24]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [25]:
science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [26]:
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [28]:
# 사용자 정의 함수 사용하기
# LangChain 공식 도큐먼트에서 권장하는 방법.
# 서로 다른 출력 간의 라우팅을 위해 사용자 정의 함수를 RunnableLambda 로 래핑하여 활용할 수도 있습니다.

In [30]:
def route(info):
    if "수학" in info["topic"].lower():     # 주제에 "수학"이 포함되어 있는 경우
        return math_chain                   # datascience_chain 리턴
                                            
    elif "과학" in info["topic"].lower():   # 주제에 "과학"이 포함되어 있는 경우        
        return science_chain                # art_chain 리턴
    
    else:                                   # 그 외의 경우        
        return general_chain                # general_chain 리턴

In [32]:
full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(        
        route           # 경로를 지정하는 함수를 인자로 넣는다.
    )   
    | StrOutputParser()
)

In [33]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

"깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 미분과 적분이라는 두 가지 주요 개념으로 구성되어 있습니다. 미분은 함수의 변화율을 구하는 과정으로, 특정 점에서의 기울기나 순간적인 변화를 분석하는 데 사용됩니다. 적분은 함수의 전체적인 면적이나 누적량을 구하는 방법으로, 주어진 구간에서의 합을 찾는 과정입니다. \n\n미적분은 물리학, 공학, 경제학 등 다양한 분야에서 매우 중요한 역할을 하며, 연속적인 변화와 그에 따른 양의 변화를 이해하는 데 핵심적인 도구입니다. 이 두 가지 개념은 서로 깊은 관계가 있으며, 기본적으로 미적분학의 기본 정리인 '미분과 적분의 관계'를 통해 연결됩니다."

In [34]:
# 과학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력은 물체 간의 인력으로, 모든 물체는 그 질량에 비례하여 서로를 끌어당기는 힘이 작용한다고 하셨습니다. 이 힘은 물체의 질량이 클수록, 그리고 물체 간의 거리가 가까울수록 강해집니다. 뉴턴의 만유인력 법칙에 따르면, 두 물체 사이의 중력은 두 물체의 질량의 곱에 비례하고, 그들 사이의 거리의 제곱에 반비례합니다. 이 원리는 지구가 사물과 사람을 끌어당기는 이유를 설명하며, 우주에서 행성들이 궤도를 유지하는 원리이기도 합니다.'

In [36]:

# 기타 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "대한민국의 수도는?"})

'서울입니다.'